In [1]:
import random

def _generate(label, a, b):
    num_states = random.choice(range(a, b + 1))
    states = []
    for i in range(num_states):
        tf = "false" if random.random() < 0.5 else "true"
        state = "S{}: S{} is a {} statement.".format(i + 1, i, tf)
        states += [state]
        label = 1 - label if tf == "false" else label

    return states, label

In [12]:
a, b = 0, 4
random.seed(1892)
train = []
with open("raw/train_raw.tsv", "r") as f:
    for i, line in enumerate(f):
        if i == 0:
            continue
        sent, label = line.strip().split("\t")
        sent = "S0: " + sent
        states, label = _generate(eval(label), a, b)
        sent = " [n] ".join([sent] + states)
        question = "Is S{} true or false?".format(len(states))
        train += [[sent, question, label]]
test = []
with open("raw/test_raw.tsv", "r") as f:
    for i, line in enumerate(f):
        if i == 0:
            continue
        sent, label = line.strip().split("\t")
        sent = "S0: " + sent
        states, label = _generate(eval(label), a, b)
        sent = " [n] ".join([sent] + states)
        question = "Is S{} true or false?".format(len(states))
        test += [[sent, question, label]]

In [13]:
with open("u/train_u{}{}.tsv".format(a, b), "w") as f:
    f.write("\t".join(["sentence", "question", "label"]) + "\n")
    for line in train:
        f.write("\t".join([line[0], line[1], str(line[2])]) + "\n")
with open("u/test_u{}{}.tsv".format(a, b), "w") as f:
    f.write("\t".join(["sentence", "question", "label"]) + "\n")
    for line in test:
        f.write("\t".join([line[0], line[1], str(line[2])]) + "\n")

In [14]:
import random

def _generate_plus(label, a, b):
    num_states = random.choice(range(a, b + 1))
    _label = label
    states = []
    labels = []
    j = 0
    if num_states > 1:
        j = random.choice(range(1, num_states))
    for i in range(num_states):
        tf = "false" if random.random() < 0.5 else "true"
        state = "S{}: S{} is a {} statement.".format(i + 1, i, tf)
        if j > 0 and i == j:
            t = random.choice(range(0, i))
            label_t = labels[t - 1] if t > 0 else _label
            if random.random() < 0.5:
                op = "or"
                state = "S{}: Either S{} or S{} is a {} statement.".format(i + 1, i, t, tf)
                if tf == "false":
                    if label == 1 or label_t == 1:
                        label = 0
                    else:
                        label = 1
                else:
                    if label == 0 or label_t == 0:
                        label = 0
                    else:
                        label = 1
            else:
                op = "and"
                state = "S{}: Both S{} and S{} are {} statements.".format(i + 1, i, t, tf)
                if tf == "false":
                    if label == 1 and label_t == 1:
                        label = 0
                    else:
                        label = 1
                else:
                    if label == 0 and label_t == 0:
                        label = 0
                    else:
                        label = 1
        else:
            label = 1 - label if tf == "false" else label
        states += [state]
        labels += [label]

    return states, labels

In [17]:
a, b = 0, 4
random.seed(1892)
train = []
with open("raw/train_raw.tsv", "r") as f:
    for i, line in enumerate(f):
        if i == 0:
            continue
        sent, label = line.strip().split("\t")
        sent = "S0: " + sent
        states, labels = _generate_plus(eval(label), a, b)
        label = labels[-1] if labels else eval(label)
        sent = " [n] ".join([sent] + states)
        question = "Is S{} true or false?".format(len(states))
        train += [[sent, question, label]]
test = []
with open("raw/test_raw.tsv", "r") as f:
    for i, line in enumerate(f):
        if i == 0:
            continue
        sent, label = line.strip().split("\t")
        sent = "S0: " + sent
        states, labels = _generate_plus(eval(label), a, b)
        label = labels[-1] if labels else eval(label)
        sent = " [n] ".join([sent] + states)
        question = "Is S{} true or false?".format(len(states))
        test += [[sent, question, label]]

In [18]:
with open("u/train_a{}{}.tsv".format(a, b), "w") as f:
    f.write("\t".join(["sentence", "question", "label"]) + "\n")
    for line in train:
        f.write("\t".join([line[0], line[1], str(line[2])]) + "\n")
with open("u/test_a{}{}.tsv".format(a, b), "w") as f:
    f.write("\t".join(["sentence", "question", "label"]) + "\n")
    for line in test:
        f.write("\t".join([line[0], line[1], str(line[2])]) + "\n")